In [ ]:
!pip install transformers
!pip install sacremoses

In [ ]:
!pip uninstall numba
!pip install umap-learn
!pip install -U numba

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
from sklearn.model_selection import cross_val_score, train_test_split
import joblib
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt
import torch
from transformers import HerbertTokenizer, RobertaModel, AutoTokenizer, BertModel

import umap
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
def get_data_set(labels, df):
  idxs = []
  for label in labels:
    idxs_for_label, = np.where(df['Label'] == label)
    for idx in idxs_for_label:
      idxs.append(idx)

  new_df = df.iloc[idxs]
  new_df = new_df.sample(frac = 1).reset_index(drop=True)
  return new_df

In [ ]:
def print_classes(df):
  y = df["Label"]
  authors = {}
  num_classes = len(df['Label'].unique())
  for label in range(0, num_classes):
    i, = np.where(y == label)
    authors['{}'.format(df['Author-short'][i[0]])] = label

  return authors

In [ ]:
def make_tokens(df, model):
  model_name, tokenizer, model = model
  tokens = {}

  df_tokens = pd.DataFrame()
  tokenize = lambda sent: tokenizer.encode_plus(sent, max_length=512, padding='max_length', truncation=True)
  df_tokens['tokens'] = df['Text'].map(tokenize)
  df_tokens['input_ids'] = df_tokens['tokens'].map(lambda t: t['input_ids'] )
  df_tokens['token_type_ids'] = df_tokens['tokens'].map(lambda t: t['token_type_ids'] )
  df_tokens['attention_mask'] = df_tokens['tokens'].map(lambda t: t['attention_mask'] )


  input_ids = np.stack(df_tokens['input_ids'])
  token_type_ids = np.stack(df_tokens['token_type_ids'])
  attention_mask = np.stack(df_tokens['attention_mask'])

  inputs = {"input_ids":torch.tensor(input_ids),"token_type_ids":torch.tensor(token_type_ids),"attention_mask":torch.tensor(attention_mask)}

  return df_tokens, inputs


In [ ]:
df_tokens, inputs = make_tokens(df_raw, herbert)

In [ ]:
outputs = model(**inputs)

In [ ]:
#dzielenie tokenów na 9 porcji
X_stack = []
embedded = {}
model_name, tokenizer, model = herbert

for idx in tqdm(range(0,400)):
  x = 0

  for i in range(0,9):

    y = x+100
    if(i>7):
      y = 512

    input_ids = np.stack(df_tokens["input_ids"].iloc[idx:idx+1])
    token_type_ids = np.stack(df_tokens["token_type_ids"].iloc[idx:idx+1])
    attention_mask = np.stack(df_tokens["attention_mask"].loc[idx:idx+1])

    input_ids = np.array([input_ids[0][x:y]])
    token_type_ids = np.array([token_type_ids[0][x:y]])
    attention_mask = np.array([attention_mask[0][x:y]])

    x = x + 60

    inputs = {"input_ids":torch.tensor(input_ids),"token_type_ids":torch.tensor(token_type_ids),"attention_mask":torch.tensor(attention_mask)}

    single_poem_output = model(**inputs)
    X_single_poem = single_poem_output[0][:,0,:].detach().numpy()
    X_stack.append(X_single_poem[0])
    embedded[idx,i] = X_single_poem[0], df_raw['Label'][idx]


  df_embedded = pd.DataFrame.from_dict(embedded,  orient='index', columns=['{}_embedding'.format(model_name), 'label'])



In [ ]:
def make_embedding(df, model):

  X_stack = []
  model_name, tokenizer, model = model
  embedded = {}
  tokens = {}
  num_idxs = df.shape[0]
  for idx in tqdm(range(0,num_idxs)):
    single_poem_input = df['Text'][idx]
    inputs = tokenizer.batch_encode_plus([single_poem_input], max_length = 512, padding="longest", add_special_tokens=True, return_tensors="pt",)
    single_poem_output = model(**inputs)
    X_single_poem = single_poem_output[0][:,0,:].detach().numpy()
    X_stack.append(X_single_poem[0])

    embedded[idx] = X_single_poem[0], df['Label'][idx]

  df_embedded = pd.DataFrame.from_dict(embedded,  orient='index', columns=['{}_embedding'.format(model_name), 'label'])

  return df_embedded

In [ ]:
def get_X_y(df):

  X = np.stack(df['Herbert_embedding'])
  y = df['Label']

  #jeśli jest mniej niż 8 klas:
  if len(df['Label'].unique()) < 8:
    y = df ['Label'].factorize()[0]

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

  print(X.shape)

  return X, y, X_train, X_test, y_train, y_test,  X_val, y_val

In [ ]:
def normalize_data(X):

  X_normalized = np.zeros((X.shape[0],X.shape[1]))

  for idx in range(0,400):
    X_normalized[idx,:] = (X[idx,:] - np.mean(X[idx,:]))/ np.std(X[idx,:])

  return X_normalized

In [ ]:
herbert = ["Herbert", HerbertTokenizer.from_pretrained("allegro/herbert-large-cased"), RobertaModel.from_pretrained("allegro/herbert-large-cased")]
#bert = ["Bert", AutoTokenizer.from_pretrained("bert-base-uncased"), BertModel.from_pretrained("bert-base-uncased")]

In [ ]:
df_raw = pd.read_csv('/content/drive/MyDrive/wiersze_do_BERT_light.csv', ";")
df_raw .columns

In [ ]:
df_raw  = df_raw .drop(columns = ['Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'])
df_raw.shape

In [ ]:
df_orginal = pd.DataFrame
df_orginal = pd.concat([df_raw['Text'],df_raw['Label'],df_raw['Author-short']], axis=1)

In [ ]:
df_orginal

In [ ]:
df_women = df_raw[200:].reset_index(drop=True)
df_men = df_raw[:200].reset_index(drop=True)
#df_women = df_women.sample(frac = 1).reset_index(drop=True)
#df_men = df_men.sample(frac = 1).reset_index(drop=True)

In [ ]:
labels = [4,5,6,7]
df = get_data_set(labels, df_raw)
print("Number of classes: {}".format(len(df['Label'].unique())))
print("Shape of new data set: {}".format(df.shape))

In [ ]:
embedding = make_embedding(df_orginal, herbert)

In [ ]:
df_orginal = pd.concat([df_orginal, embedding['Herbert_embedding']], axis=1)
df_orginal

In [ ]:
classes = print_classes(df_orginal)
classes

In [ ]:
X, y, _,_,_,_,_,_ = get_X_y(df_orginal)

In [ ]:
# Umap
df_umap = pd.DataFrame()
df_umap["y"] = df_orginal['Author-short']
data_type = 'all'
num_classes = len(classes)
n_neighbors = [10]
min_distnces= [0.1]
n_components=2
metric='euclidean'

for n_neighbor in n_neighbors:
  for min_dist in min_distnces:
    reducer = umap.UMAP(n_neighbors=n_neighbor,
            min_dist=min_dist,
            n_components=n_components,
            metric=metric)
    scaled_X = StandardScaler().fit_transform(X)
    embed = reducer.fit_transform(scaled_X)

    df_umap["comp-1"] = embed[:,0]
    df_umap["comp-2"] = embed[:,1]

    plt.figure(figsize = (10,10))
    sns.scatterplot(x="comp-1", y="comp-2", hue=df_umap.y.tolist(),
                   palette=sns.color_palette("Set2", num_classes),
                   data=df_umap, s=100).set(title="Poems data Umap projection | Data type: {} | N_neighbors: {} | Distance: {}".format(data_type, n_neighbor, min_dist))


plt.savefig('/content/figs/umap_{}.png'.format(data_type))
files.download('/content/figs/umap_{}.png'.format(data_type))

In [ ]:
# 3D PCA
data_type = 'all'
scaler = StandardScaler()
scaler.fit(X)
scaled_X = scaler.transform(X)
pca = PCA(n_components=3)
pca.fit(scaled_X)
pca_X = pca.transform(scaled_X)

In [ ]:
Xax = pca_X[:,0]
Yax = pca_X[:,1]
Zax = pca_X[:,2]

cdict = {0:'cyan',1:'red',2:'blue',3:'green',4:'yellow',5:'violet',6:'orange',7:'brown'}
label = {0:'K. K. Baczyński',1:'A. Mickiewicz',2:'J. Kochanowki',3:'Cz. Miłosz',4:'W. Szymborska',5:'H. Poświatowska',6:'M. P. Jasnorzewska',7:'E. Lipska'}


fig = plt.figure(figsize=(14,9))
ax = fig.add_subplot(111,
                     projection='3d')

for l in np.unique(y):
 ix=np.where(y==l)
 ax.scatter(Xax[ix],
            Yax[ix],
            Zax[ix],
            c=cdict[l],
            s=60,
           label=label[l])

ax.set_xlabel("PC1",
              fontsize=12)
ax.set_ylabel("PC2",
              fontsize=12)
ax.set_zlabel("PC3",
              fontsize=12)

ax.view_init(30, 140)
ax.legend()
plt.title("Poems data 3D PCA projection | Data type: {}".format(data_type))
plt.show()

plt.savefig('/content/figs/pca_3D_{}.png'.format(data_type))
files.download('/content/figs/pca_3D_{}.png'.format(data_type))

In [ ]:
X_normalized = normalize_data(X)
print(X_normalized)

In [ ]:
distance_euclidean_X_normalized = np.zeros((X_normalized.shape[0], X_normalized.shape[0]))
distance_cosinus_X_normalized = np.zeros((X_normalized.shape[0], X_normalized.shape[0]))

for i in range(0,distance_euclidean_X_normalized.shape[0]):
  x_normalized_i = X_normalized[i,:]
  for j in range(0,distance_euclidean_X_normalized.shape[0]):
    x_normalized_j = X_normalized[j,:]
    distance_euclidean_X_normalized[i,j] = np.sqrt(np.sum( np.abs(x_normalized_i - x_normalized_j)**2))
    distance_cosinus_X_normalized[i,j] = np.dot(x_normalized_i, x_normalized_j)/1024


In [ ]:
distances = {"Euclidean distance":distance_euclidean_X_normalized, "Cosinus distance":distance_cosinus_X_normalized}
data_type = "all"
fig, ax = plt.subplots(1,2, figsize=(20,10), sharey='row')
im = ax[0].imshow(distance_euclidean_X_normalized)
ax[0].set_title('Euclidean distance', fontsize=20)
im2 = ax[1].imshow(distance_cosinus_X_normalized)
ax[1].set_title('Cosinus distance', fontsize=20)
fig.colorbar(im, ax=ax[0])
fig.colorbar(im2, ax=ax[1])

fig.savefig('/content/figs/euclidean_and_cosinus.png')
files.download('/content/figs/euclidean_and_cosinus.png')


In [ ]:
fig2, axes = plt.subplots(1, 2, figsize=(20,10), sharey='row')

  for i, (key, cm) in enumerate(cms.items()):

    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=cls)
    disp.plot(ax=axes[i], xticks_rotation=45)
    disp.ax_.set_title("{} | Model: Neural Network | Data type: {} | Acc: {}".format(key, data_type, round(score_avrg,2)))
    disp.im_.colorbar.remove()
    disp.ax_.set_xlabel('')
    disp.ax_.set_ylabel('')


  fig2.text(0.40, 0.1, 'Predicted label', ha='left')
  plt.subplots_adjust(wspace=0.40, hspace=0.1)

  fig2.colorbar(disp.im_, ax=axes)
  plt.show()

  plt.gcf().set_size_inches(10, 5)
  fig2.savefig('/content/figs/avrg_neural_network_{}_scheme_{}_fraction_{}.png'.format(data_type, scheme, fraction), dpi=200)
  files.download('/content/figs/avrg_neural_network_{}_scheme_{}_fraction_{}.png'.format(data_type, scheme, fraction))

In [ ]:
plt.imshow(distance_cosinus_X_normalized)
plt.colorbar()
plt.title('Distance between vectors | data type: men')

In [ ]:
M_men = np.zeros((200,200))

for i in range(0,200):
  x_i = embedded_men['Herbert_embedding'][i]
  for j in range(0,200):
    x_j = embedded_men['Herbert_embedding'][j]

    #scalar_product = np.dot(x_i, x_j)
    #M[i,j] = scalar_product
    M_men[i,j] = np.sqrt(np.sum( np.abs(x_i - x_j)**2))
plt.imshow(M_men)
plt.colorbar()
plt.title('Distance between vectors | data type: men')


plt.savefig('/content/odległość_wektorów_po_ombeddingu_men.png')
files.download('/content/odległość_wektorów_po_ombeddingu_men.png')

In [ ]:
M_women = np.zeros((200,200))

for i in range(0,200):
  x_i = embedded_women['Herbert_embedding'][i]
  for j in range(0,200):
    x_j = embedded_women['Herbert_embedding'][j]

    #scalar_product = np.dot(x_i, x_j)
    #M[i,j] = scalar_product
    M_women[i,j] = np.sqrt(np.sum( np.abs(x_i - x_j)**2))
plt.imshow(M_women)
plt.colorbar()

plt.title('Distance between vectors | data type: women')


plt.savefig('/content/odległość_wektorów_po_ombeddingu_women.png')
files.download('/content/odległość_wektorów_po_ombeddingu_women.png')

In [ ]:
M = np.zeros((400,400))
for i in range(0,400):
  x_i = embedded_all['Herbert_embedding'][i]
  for j in range(0,400):
    x_j = embedded_all['Herbert_embedding'][j]

    #scalar_product = np.dot(x_i, x_j)
    #M[i,j] = scalar_product
    M[i,j] = np.sqrt(np.sum( np.abs(x_i - x_j)**2))
#M = M/1024.

In [ ]:
plt.imshow(M)
plt.colorbar()
plt.title('Distance between vectors | data type: all')


plt.savefig('/content/odległość_wektorów_po_ombeddingu_all.png')
files.download('/content/odległość_wektorów_po_ombeddingu_all.png')

In [ ]:
classes = print_classes(df)
classes

In [ ]:
df_pca = pd.DataFrame()
df_pca["y"] = df_men['Author-short']
data_type = 'men'
num_classes = len(classes)

plt.figure(figsize = (10,10))
pca = PCA(n_components=2).fit_transform(X, y)

df_pca["comp-1"] = pca[:,0]
df_pca["comp-2"] = pca[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue=df_pca.y.tolist(),
                palette=sns.color_palette("Set2", num_classes),
                data=df_pca, s=100).set(title="Poems data PCA projection | data_type: {}".format(data_type))

plt.savefig('/content/figs/pca_{}.png'.format(data_type))
files.download('/content/figs/pca_{}.png'.format(data_type))

In [ ]:
df_tsne = pd.DataFrame()
df_tsne["y"] = df_light['Author-short']
data_type = 'men'
num_classes = len(classes)

perps = [5, 10, 15, 20, 25, 30, 35, 40]

for perp in perps:

  plt.figure(figsize = (10,10))
  z = TSNE(n_components=2, learning_rate='auto', init='pca', perplexity=perp).fit_transform(X, y)

  df_tsne["comp-1{}".format(perp)] = z[:,0]
  df_tsne["comp-2{}".format(perp)] = z[:,1]

  sns.scatterplot(x="comp-1{}".format(perp), y="comp-2{}".format(perp), hue=df_tsne.y.tolist(),
                palette=sns.color_palette("Set2", num_classes),
                data=df_tsne, s=100).set(title="Poems data t-SNE projection | perplexity: {} | data_type: {}".format(perp, data_type))

  plt.savefig('/content/figs/tsne_{}_{}_perplexity.png'.format(data_type, perp))
  files.download('/content/figs/tsne_{}_{}_perplexity.png'.format(data_type, perp))